# Stateful Elasticsearch Feedback Workflow for Metrics Server
In this example we will add statistical performance metrics capabilities by levering the Seldon metrics server with persistence through the elasticsearch setup.

Dependencies
* Seldon Core installed
* Ingress provider (Istio or Ambassador)
* Install [Elasticsearch for the Seldon Core Logging](https://docs.seldon.io/projects/seldon-core/en/latest/analytics/logging.html)
* KNative eventing v0.11.0
* KNative serving v0.11.1 (optional)

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

Ambassador:

    kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080

Istio:

    kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80




In [76]:
%%writefile requirements-dev.txt
elasticsearch==7.9.1

Overwriting requirements-dev.txt


In [77]:
!pip install -r requirements-dev.txt

     |████████████████████████████████| 225kB 1.1MB/s eta 0:00:01
  Found existing installation: elasticsearch 7.5.1
    Uninstalling elasticsearch-7.5.1:
      Successfully uninstalled elasticsearch-7.5.1


In [5]:
!kubectl create namespace seldon || echo "namespace already created"

namespace/seldon created


In [6]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "docker-desktop" modified.


In [65]:
!mkdir -p config

Setting up Knative eventing routing for request logger 

In [37]:
!kubectl label namespace seldon-logs knative-eventing-injection=enabled --overwrite=true

namespace/seldon labeled


Adding payload request logger component for redirection of logs

In [17]:
%%bash
kubectl apply -f - << END
apiVersion: eventing.knative.dev/v1alpha1
kind: Trigger
metadata:
 name: seldon-request-logger-trigger
 namespace: seldon-logs
spec:
 subscriber:
   ref:
     apiVersion: v1
     kind: Service
     name: seldon-request-logger
END

kubectl apply -f - << END
apiVersion: apps/v1
kind: Deployment
metadata:
 name: seldon-request-logger
 namespace: seldon-logs
 labels:
   app: seldon-request-logger
spec:
 replicas: 1
 selector:
   matchLabels:
     app: seldon-request-logger
 template:
   metadata:
     labels:
       app: seldon-request-logger
   spec:
     containers:
       - name: user-container
         image: docker.io/seldonio/seldon-request-logger:1.4.0-dev
         imagePullPolicy: IfNotPresent
         env:
           - name: ELASTICSEARCH_HOST
             value: "elasticsearch-master.seldon-logs.svc.cluster.local"
           - name: ELASTICSEARCH_PORT
             value: "9200"
END

kubectl apply -f - << END
apiVersion: v1
kind: Service
metadata:
 name: seldon-request-logger
 namespace: seldon-logs
spec:
 selector:
   app: seldon-request-logger
 ports:
   - protocol: TCP
     port: 80
     targetPort: 8080
END

trigger.eventing.knative.dev/seldon-request-logger-trigger unchanged
deployment.apps/seldon-request-logger configured
service/seldon-request-logger unchanged


### Create a simple model
We create a multiclass classification model - iris classifier.

The iris classifier takes an input array, and returns the prediction of the 4 classes.

The prediction can be done as numeric or as a probability array.

In [51]:
%%bash
kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: multiclass-model
spec:
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
      logger:
        url: http://default-broker.seldon-logs.svc.cluster.local:80/
        mode: all
    name: default
    replicas: 1
END

seldondeployment.machinelearning.seldon.io/multiclass-model configured


In [18]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=multiclass-model -o jsonpath='{.items[0].metadata.name}')

deployment "multiclass-model-default-0-classifier" successfully rolled out


### Send Prediction Request

In [19]:
import requests
url = "http://localhost:80/seldon/seldon/multiclass-model/api/v1.0"

In [22]:
pred_req_1 = {"data":{"ndarray":[[1,2,3,4]]}}
pred_resp_1 = requests.post(f"{url}/predictions", json=pred_req_1)
print(pred_resp_1.json())
assert(len(pred_resp_1.json()["data"]["ndarray"][0])==3)

{'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[0.0006985194531162841, 0.003668039039435755, 0.9956334415074478]]}, 'meta': {}}


### Check data in Elasticsearch
We'll be able to check the elasticsearch through the service or the pods in our cluster.

In [23]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['http://localhost:9200'])

See the indices that have been created

In [24]:
es.indices.get_alias("*")

{'inference-log-seldon-seldon-multiclass-model-default': {'aliases': {}}}

Look at the data that is stored in the elasticsearch index

In [25]:
res = es.search(index="inference-log-seldon-seldon-multiclass-model-default", body={"query": {"match_all": {}}})
print("Logged Request:")
print(res["hits"]["hits"][0]["_source"]["request"])
print("\nLogged Response:")
print(res["hits"]["hits"][0]["_source"]["response"])

Logged Request:
{'payload': {'data': {'ndarray': [[1, 2, 3, 4]]}, 'meta': {'puid': 'hello'}}, 'dataType': 'tabular', 'elements': {}, 'instance': [1.0, 2.0, 3.0, 4.0], 'ce-time': '2020-11-02T13:06:44.024402323Z', 'ce-source': 'http::8000'}

Logged Response:
{'ce-source': 'http::8000', 'instance': [0.0006985194531162841, 0.003668039039435755, 0.9956334415074478], 'names': ['t:0', 't:1', 't:2'], 'payload': {'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[0.0006985194531162841, 0.003668039039435755, 0.9956334415074478]]}, 'meta': {'puid': 'hello'}}, 'dataType': 'tabular', 'elements': {'t:1': [0.003668039039435755], 't:0': [0.0006985194531162841], 't:2': [0.9956334415074478]}, 'ce-time': '2020-11-02T13:06:44.042412223Z'}


In [26]:
res = es.get(index="inference-log-seldon-seldon-multiclass-model-default", id="7983e38c-29dc-45ff-8ffb-77252e7ac86d")
print(res["_source"]["response"])

{'ce-source': 'http::8000', 'instance': [0.0006985194531162841, 0.003668039039435755, 0.9956334415074478], 'names': ['t:0', 't:1', 't:2'], 'payload': {'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[0.0006985194531162841, 0.003668039039435755, 0.9956334415074478]]}, 'meta': {'puid': 'hello'}}, 'dataType': 'tabular', 'elements': {'t:1': [0.003668039039435755], 't:0': [0.0006985194531162841], 't:2': [0.9956334415074478]}, 'ce-time': '2020-11-02T13:06:44.042412223Z'}


### Send feedback

We can now send the correction, or the truth value of the prediction.

For this we'll need to send the UUID of the feedback request to ensure it's added to the correct index.

In [27]:
puid_seldon_1 = pred_resp_1.headers.get("seldon-puid")

print(puid_seldon_1)

7f9e69df-6601-4947-b30b-e05a12f5726b


We can also be able to add extra metadata, such as the user providing the feedback, date, time, etc.

In [28]:
feedback_tags_1 = {
    "user": "Seldon Admin",
    "date": "11/07/2020"
}

And finally we can put together the feedback request.

In [29]:
feedback_req_1 = {
    "reward": 0,
    "truth": {
        'data': {
            'names': ['t:0', 't:1', 't:2'], 
            'ndarray': [[0, 0, 1]]
        },
        "meta": {
            "tags": feedback_tags_1
        }
    }
}

And send the feedback request

In [30]:
feedback_resp_1 = requests.post(f"{url}/feedback", json=feedback_req_1, headers={"seldon-puid": puid_seldon_1})
print(feedback_resp_1)

<Response [200]>


Check that feedback has been received and stored in the Elasticsearch index

In [31]:
res = es.search(index="inference-log-seldon-seldon-multiclass-model-default", body={"query": {"match_all": {}}})

print(res["hits"]["hits"][-1]["_source"]["feedback"])

{'reward': 0, 'ce-source': 'http::8000', 'truth': {'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[0, 0, 1]]}, 'meta': {'tags': {'date': '11/07/2020', 'user': 'Seldon Admin'}}}, 'ce-time': '2020-11-02T18:26:22.03643351Z'}


### Deploying Metrics Server

Now we'll be able to see how the metrics server makes use of this infrastructure patterns to provide real time performance metrics.

In [74]:
%%writefile config/multiclass-deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: seldon-multiclass-model-metrics
  labels:
    app: seldon-multiclass-model-metrics
spec:
  replicas: 1
  selector:
    matchLabels:
      app: seldon-multiclass-model-metrics
  template:
    metadata:
      labels:
        app: seldon-multiclass-model-metrics
    spec:
      securityContext:
          runAsUser: 8888
      containers:
      - name: user-container
        image: seldonio/alibi-detect-server:1.4.0-dev
        imagePullPolicy: IfNotPresent
        args:
        - --model_name
        - multiclassserver
        - --http_port
        - '8080'
        - --protocol
        - seldonfeedback.http
        - --storage_uri
        - "adserver.cm_models.multiclass_one_hot.MulticlassOneHot"
        - --reply_url
        - http://message-dumper.default        
        - --event_type
        - io.seldon.serving.feedback.metrics
        - --event_source
        - io.seldon.serving.feedback
        - --elasticsearch_uri
        - http://elasticsearch-master.seldon-logs:9200
        - MetricsServer
        env:
        - name: "SELDON_DEPLOYMENT_ID"
          value: "multiclass-model"
        - name: "PREDICTIVE_UNIT_ID"
          value: "classifier"
        - name: "PREDICTIVE_UNIT_IMAGE"
          value: "alibi-detect-server:1.3.0-dev"
        - name: "PREDICTOR_ID"
          value: "default"
---
apiVersion: v1
kind: Service
metadata:
  name: seldon-multiclass-model-metrics
  labels:
    app: seldon-multiclass-model-metrics
spec:
  selector:
    app: seldon-multiclass-model-metrics
  ports:
    - protocol: TCP
      port: 80
      targetPort: 8080

Overwriting config/multiclass-deployment.yaml


In [75]:
!kubectl apply -n seldon -f config/multiclass-deployment.yaml

deployment.apps/seldon-multiclass-model-metrics configured
service/seldon-multiclass-model-metrics unchanged


In [15]:
!kubectl rollout status deploy/seldon-multiclass-model-metrics

deployment "seldon-multiclass-model-metrics" successfully rolled out


### Trigger for metrics server

The trigger will be created in the seldon-logs namespace as that is where the initial trigger will be sent to.

In [52]:
%%bash

kubectl apply -f - << END
apiVersion: eventing.knative.dev/v1alpha1
kind: Trigger
metadata:
  name: multiclass-model-metrics-trigger
  namespace: seldon-logs
spec:
  filter:
    sourceAndType:
      type: io.seldon.serving.feedback
  subscriber:
    uri: http://seldon-multiclass-model-metrics.seldon:80
END

trigger.eventing.knative.dev/multiclass-model-metrics-trigger created


In [ ]:
import time
time.sleep(20)

### (Alternative) create kservice

If you want to create a kservice, and you've installed knative eventing and knative serving, you can use the instructions below.

The value of the file `config/multiclass-service.yaml` would be:
```
apiVersion: serving.knative.dev/v1alpha1
kind: Service
metadata:
  name: seldon-multiclass-model-metrics
spec:
  template:
    metadata:
      annotations:
        autoscaling.knative.dev/minScale: "1"
    spec:
      containers:
      - image: "seldonio/alibi-detect-server:1.4.0-dev"
        args:
        - --model_name
        - multiclassserver
        - --http_port
        - '8080'
        - --protocol
        - seldonfeedback.http
        - --storage_uri
        - "adserver.cm_models.multiclass_one_hot.MulticlassOneHot"
        - --reply_url
        - http://message-dumper.default        
        - --event_type
        - io.seldon.serving.feedback.metrics
        - --event_source
        - io.seldon.serving.feedback
        - MetricsServer
        env:
        - name: "SELDON_DEPLOYMENT_ID"
          value: "multiclass-model"
        - name: "PREDICTIVE_UNIT_ID"
          value: "classifier"
        - name: "PREDICTIVE_UNIT_IMAGE"
          value: "alibi-detect-server:1.3.0-dev"
        - name: "PREDICTOR_ID"
          value: "default"
        securityContext:
            runAsUser: 8888
```

You can run the kservice with the command below:
```
kubectl apply -f config/multiclass-service.yaml
```
And then check with:

```
kubectl get kservice
```

You'll then have to create the trigger, first by creating the broker:

```
kubectl label namespace default knative-eventing-injection=enabled --overwrite=true
```

And then the trigger contents:

```
apiVersion: eventing.knative.dev/v1alpha1
kind: Trigger
metadata:
  name: multiclass-model-metrics-trigger
  namespace: default
spec:
  filter:
    sourceAndType:
      type: io.seldon.serving.feedback
  subscriber:
    ref:
      apiVersion: v1
      kind: Service
      name: seldon-multiclass-model-metrics
```

And you can run it with:
```
kubectl apply -f config/trigger.yaml
```

### Confirm empty metrics

In [43]:
!kubectl run --quiet=true -it --rm curl --image=radial/busyboxplus:curl --restart=Never -- \
    curl -v -X GET "http://seldon-multiclass-model-metrics.seldon.svc.cluster.local:80/v1/metrics" 

### Send Feedback

In [65]:
feedback_resp_1 = requests.post(f"{url}/feedback", json=feedback_req_1, headers={"seldon-puid": puid_seldon_1})
print(feedback_resp_1)

<Response [200]>


### Confirm metrics available

In [66]:
!kubectl run --quiet=true -it --rm curl --image=radial/busyboxplus:curl --restart=Never -- \
    curl -v -X GET "http://seldon-multiclass-model-metrics.seldon:80/v1/metrics" 

### Cleanup

In [96]:
!kubectl delete -n seldon -f config/multiclass-deployment.yaml

deployment.apps "seldon-multiclass-model-metrics" deleted
service "seldon-multiclass-model-metrics" deleted


In [97]:
!kubectl delete sdep multiclass-model

seldondeployment.machinelearning.seldon.io "multiclass-model" deleted
